In [1]:
import pandas as pd
import numpy as np

<font size = 8>Part 1</font>

Import data frame using Pandas. First data frame from the site is the correct one.

In [34]:
df = pd.DataFrame
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
df = df[0]

In [35]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Filter out any rows with unassigned borough

In [36]:
df=df[df['Borough']!='Not assigned']

Separate neighborhoods with a comma. No examples where this is true in current wiki page.

In [37]:
df['Neighborhood'] = df.groupby(['Postal Code'])['Neighborhood'].transform(lambda x: ','.join(x))
df.drop_duplicates()
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


If neighborhood is not assigned, replace neighborhood with Borough. No examples where this is true in current wiki page.

In [38]:
df['Neighborhood'] = np.where((df['Neighborhood'] == 'Not assigned'),df['Borough'], df['Neighborhood'])

In [39]:
df.shape

(103, 3)

<font size = 8>Part 2</font>

Import geospatial data from CSV

In [40]:
latlong = pd.read_csv('http://cocl.us/Geospatial_data')

Join new geospatial data to dataframe

In [41]:
df2 = df.merge(latlong, on = 'Postal Code')

In [42]:
df2.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


<font size = 8>Part 3</font>

In [43]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Normalize data for clustering

In [44]:
X = df2.values[:,3:5]
cluster_dataset = StandardScaler().fit_transform(X)

Run k-means clustering

In [45]:
# run k-means clustering
kclusters = 5
k_means = KMeans(n_clusters=kclusters, random_state=0).fit(cluster_dataset)
k_means.fit(cluster_dataset)
labels = k_means.labels_
df2['Cluster Labels'] = k_means.labels_

Get coordinates of Toronto

In [46]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [47]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium # map rendering library

Create map

In [48]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df2['Latitude'], df2['Longitude'], df2['Neighborhood'], df2['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters